<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# AI-First Finance

**Recurrent Neural Networks**

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance.git
import sys
sys.path.append('ai_in_finance')


In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')

In [ ]:
import cufflinks as cf
cf.set_config_file(offline=True)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
np.random.seed(100)
import tensorflow as tf
tf.random.set_seed(100)

## First Example

In [ ]:
a = np.arange(100)
a

In [ ]:
a = a.reshape((len(a), -1))

In [ ]:
a.shape

In [ ]:
a[:5]

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
lags = 3

In [ ]:
g = TimeseriesGenerator(a, a, length=lags, batch_size=5)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense

In [ ]:
model = Sequential()
model.add(SimpleRNN(100, activation='relu', input_shape=(lags, 1)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
model.summary()

In [ ]:
%%time
model.fit(g, epochs=150, steps_per_epoch=10, verbose=False);

In [ ]:
res = pd.DataFrame(model.history.history)

In [ ]:
res.tail(3)

In [ ]:
ax = res.plot(figsize=(10, 6), style=['--', '--'])

In [ ]:
# make a one step prediction in-sample
x = np.array([21, 22, 23]).reshape((1, lags, 1))
y = model.predict(x, verbose=True)
y

In [ ]:
y.round()

In [ ]:
int(round(y[0, 0]))

In [ ]:
# make a one step prediction in-sample
x = np.array([87, 88, 89]).reshape((1, lags, 1))
y = model.predict(x, verbose=True)
y

In [ ]:
y.round()

In [ ]:
# make a one step prediction out of sample
x = np.array([187, 188, 189]).reshape((1, lags, 1))
y = model.predict(x, verbose=True)
y

In [ ]:
y.round()

In [ ]:
# make a one step prediction out of sample
x = np.array([1187, 1188, 1189]).reshape((1, lags, 1))
y = model.predict(x, verbose=True)
y

In [ ]:
y.round()

## Second Example

In [ ]:
def transform(x):
    y = 0.05 * x ** 2 + 0.2 * x + np.sin(x) + 5
    y += np.random.standard_normal(len(x)) * 0.2
    return y

In [ ]:
x = np.linspace(-2 * np.pi, 2 * np.pi, 500)
a = transform(x)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, a);

In [ ]:
a = a.reshape((len(a), -1))

In [ ]:
a[:5]

In [ ]:
lags = 5

In [ ]:
g = TimeseriesGenerator(a, a, length=lags, batch_size=5)

In [ ]:
# for _ in g:
#     print(_)

In [ ]:
model = Sequential()
model.add(SimpleRNN(500, activation='relu', input_shape=(lags, 1)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
model.summary()

In [ ]:
%%time
model.fit(g, epochs=500, steps_per_epoch=10, verbose=False);

In [ ]:
res = pd.DataFrame(model.history.history)

In [ ]:
res.tail(3)

In [ ]:
ax = res.plot(figsize=(10, 6), style=['--', '--'])

In [ ]:
x = np.linspace(-6 * np.pi, 6 * np.pi, 1000)
d = transform(x)

In [ ]:
len(d)

In [ ]:
lags=5

In [ ]:
gd = TimeseriesGenerator(d, d, length=lags, batch_size=len(d))

In [ ]:
#list(gd)[0][0].shape

In [ ]:
#f = list(gd)[0][0].reshape((len(d) - lags, lags, 1))

In [ ]:
y = model.predict(gd, verbose=True)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x[lags:], d[lags:], label='data')
plt.plot(x[lags:], y, 'r.', label='pred')
plt.axvline(-2 * np.pi, c='m')
plt.axvline(2 * np.pi, c='m')
plt.legend();

In [ ]:
df = pd.DataFrame({'data': d[lags:], 'pred': y.reshape(1, -1)[0]}, index=x[lags:])

In [ ]:
df.iplot()

## Financial Data

### The Data

In [ ]:
raw = pd.read_csv('http://hilpisch.com/tr_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [ ]:
symbol = 'EUR='
data = pd.DataFrame(raw[symbol])
data.info()

In [ ]:
data = data.loc[:'2014-09-30']

In [ ]:
data.info()

In [ ]:
data.iplot()

In [ ]:
split = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)

In [ ]:
train = data.iloc[:split]
val = train[-val_size:]
train = train[:-val_size]
test = data.iloc[split:].copy()

In [ ]:
lags = 5
gtr = TimeseriesGenerator(train.values, train.values, length=lags, batch_size=10)
gva = TimeseriesGenerator(val.values, val.values, length=lags, batch_size=10)

In [ ]:
# list(gtr)[0]

### Model Fitting

In [ ]:
model = Sequential()
model.add(LSTM(250, activation='relu', input_shape=(lags, 1),
               return_sequences=True))
model.add(LSTM(250, activation='relu', input_shape=(lags, 1)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
model.summary()

In [ ]:
%%time
model.fit(gtr, epochs=500, steps_per_epoch=10, verbose=False,
                    validation_data=gva);

In [ ]:
res = pd.DataFrame(model.history.history)

In [ ]:
res.tail(3)

In [ ]:
to_plot = ['mae', 'val_mae']
ax = res[to_plot].iloc[2:].plot(figsize=(10, 6), style=['--', '--', '-', '-'])
x = np.arange(len(res));
reg = np.polyfit(x, res['val_mae'], deg=2)
plt.plot(x, np.polyval(reg, x), 'r');

### Model Testing

In [ ]:
gte = TimeseriesGenerator(test.values, test.values, length=lags, batch_size=len(test))

In [ ]:
#f = list(gte)[0][0].reshape((len(test) - lags, lags, 1))

In [ ]:
y = model.predict(gte, verbose=True)

In [ ]:
data[symbol].iplot()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(test.index[lags:], test[lags:], label='data')
plt.plot(test.index[lags:], y, 'r.', label='pred')
plt.legend();

In [ ]:
df = pd.DataFrame({'data': test.values[lags:].reshape(1, -1)[0],
                   'pred': y.reshape(1, -1)[0]}, index=test.index[lags:])

In [ ]:
df.iplot()

## Patterns

### The Data

In [ ]:
symbol = 'EUR='
data = pd.DataFrame(raw[symbol])
data.info()

In [ ]:
nbins = 50

In [ ]:
split = 1900
val_size = 400

In [ ]:
train = data.iloc[1000:split]
val = train[-val_size:].copy()
train = train[:-val_size].copy()
test = data.iloc[split:].copy()

In [ ]:
adj = 0.002
mi = train.min()
ma = train.max()
bins = np.linspace(mi - adj, ma + adj, nbins).flatten()

In [ ]:
bins

In [ ]:
train['o'] = np.digitize(train[symbol], bins)
val['o'] = np.digitize(val[symbol], bins)
test['o'] = np.digitize(test[symbol], bins)

In [ ]:
train.head(5)

In [ ]:
train.iloc[:150].iplot(secondary_y='o')

In [ ]:
lags = 15
gt = TimeseriesGenerator(train['o'].values, train['o'].values,
                         length=lags, batch_size=len(train))

In [ ]:
gtr = TimeseriesGenerator(train['o'].values.reshape(-1, 1),
                          train['o'].values.reshape(-1, 1),
                          length=lags, batch_size=10)
gva = TimeseriesGenerator(val['o'].values.reshape(-1, 1),
                          val['o'].values.reshape(-1, 1),
                          length=lags, batch_size=10)

In [ ]:
# list(gtr)[0]

### Model Fitting

In [ ]:
model = Sequential()

model.add(SimpleRNN(100, activation='relu', input_shape=(lags, 1),
               return_sequences=True))    
model.add(SimpleRNN(100, activation='relu', input_shape=(lags, 1),))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
model.summary()

In [ ]:
%%time
model.fit(gtr, epochs=150, steps_per_epoch=10, verbose=False,
                   validation_data=gva);

In [ ]:
res = pd.DataFrame(model.history.history)

In [ ]:
res.tail(3)

In [ ]:
to_plot = ['mae', 'val_mae']
ax = res[to_plot].iloc[2:].plot(figsize=(10, 6), style=['--', '--', '-', '-'])
x = np.arange(len(res));
reg = np.polyfit(x, res['val_mae'], deg=2)
plt.plot(x, np.polyval(reg, x), 'r');

In [ ]:
#f = list(gt)[0][0].reshape((len(train) - lags, lags, 1))

In [ ]:
y = model.predict(gt, verbose=True)

In [ ]:
df = pd.DataFrame({'level': train['o'].values[lags:].reshape(1, -1)[0],
                   'pred': y.reshape(1, -1)[0].round()}, index=train['o'].index[lags:])

In [ ]:
df.iplot()

### Model Testing

In [ ]:
gte = TimeseriesGenerator(test['o'].values.reshape(-1, 1),
                          test['o'].values.reshape(-1, 1),
                          length=lags, batch_size=len(test))

In [ ]:
#f = list(gte)[0][0].reshape((len(test) - lags, lags, 1))

In [ ]:
y = model.predict(gte, verbose=True)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(test['o'].index[lags:], test['o'][lags:], label='level')
plt.plot(test['o'].index[lags:], y, 'r.', label='pred')
plt.legend();

In [ ]:
df = pd.DataFrame({'level': test['o'].values[lags:].reshape(1, -1)[0],
                   'pred': y.reshape(1, -1)[0].round()}, index=test['o'].index[lags:])

In [ ]:
df.iplot()

## Returns

### The Data

In [ ]:
symbol = 'EUR='
data = pd.DataFrame(raw[symbol])
data['r'] = np.log(data[symbol] / data[symbol].shift(1))
data['rs'] = np.log(data[symbol] / data[symbol].shift(1)) * 500
data['d'] = np.where(data['r'] > 0, 1, 0)

In [ ]:
lags = 10
cols = []
for lag in range(1, lags + 1):
    col = 'lag_{}'.format(lag)
    data[col] = data['r'].shift(lag)
    cols.append(col)
data.dropna(inplace=True)

In [ ]:
cols[::-1]

In [ ]:
data.head(5)

In [ ]:
len(data)

In [ ]:
split = 1600
val_size = 400

In [ ]:
train = data.iloc[:split]
val = train[-val_size:]
train = train[:-val_size]
test = data.iloc[split:].copy()

### Model Fitting

In [ ]:
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint

In [ ]:
np.random.seed(100)
tf.random.set_seed(100)

In [ ]:
model = Sequential()

# hidden layers
model.add(SimpleRNN(128, input_shape=(lags, 1),
                     return_sequences=True,
                    ))
model.add(Dropout(0.5, seed=100))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.5, seed=100))
model.add(SimpleRNN(128))

# output layer
model.add(Dense(1, activation='linear'))

# model compilation
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mse'])

In [ ]:
model.summary()

In [ ]:
callbacks = [ModelCheckpoint(filepath='../../../data/.weights.h5',
                             monitor='val_mse',
                             verbose=0,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='auto')]

In [ ]:
%%time
model.fit(train[cols[::-1]].values.reshape(len(train),lags, -1),
          train['r'].values.reshape(len(train), -1),
          epochs=25, batch_size=32, verbose=False,
          validation_data=(
              val[cols[::-1]].values.reshape(len(val),lags,-1),
              val['r'].values.reshape(len(val), -1)
          ), callbacks=callbacks);

In [ ]:
res = pd.DataFrame(model.history.history)
res.tail(3)

In [ ]:
ax = res.plot(figsize=(10, 6), style=['--', '--', '-', '-'])
x = np.arange(len(res));
reg = np.polyfit(x, res['val_mse'], deg=2)
plt.plot(x, np.polyval(reg, x), 'r');

### Backtesting

In [ ]:
model.load_weights('../../../data/.weights.h5')

In [ ]:
model.evaluate(test[cols].values.reshape(len(test), lags, -1),
               test['r'])

In [ ]:
test['p'] = model.predict(test[cols].values.reshape(len(test), lags, -1))

In [ ]:
test['p'] = np.where(test['p'] > 0, 1, -1)

In [ ]:
test['s'] = test['p'] * test['r']

In [ ]:
test[['r', 's']].sum().apply(np.exp)

In [ ]:
test['p'].value_counts()

In [ ]:
test[['r', 's']].cumsum().apply(np.exp).iplot();

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">